In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if "GPU" not in device_name:
    print("GPU device not found")
else:
    print(f'Found GPU at: {device_name}')

# Alternative way to check
print("GPU", "available (YESS!!!!)" if tf.config.list_physical_devices("GPU") else "not available :(")


2025-05-18 06:34:24.434606: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747550064.651707      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747550064.711128      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found GPU at: /device:GPU:0
GPU available (YESS!!!!)


I0000 00:00:1747550078.081342      35 gpu_device.cc:2022] Created device /device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


In [ ]:
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 9.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.2
    Uninstalling typeguard-4.4.2:
      Successfully uninstalled typeguard-4.4.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.16.1 requires typeguard<5,>=3, but you have typeguard 2.13.3 which is incompatible.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [ ]:
!pip install transformers

# **PART 2**

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvid

In [ ]:
import logging
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer, DataCollatorForTokenClassification
from datasets import load_dataset
from torch.optim import AdamW
import pandas as pd

# Suppress transformer warnings
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/tamil-bert")

from datasets import load_dataset

# Load the Tamil NER dataset
dataset = load_dataset("google/xtreme", "PAN-X.ta")
train_data = dataset['train']
validation_data = dataset['validation']
test_data = dataset['test']

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/131k [00:00<?, ?B/s]

...

validation-00000-of-00001.parquet:   0%|          | 0.00/63.3k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/62.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
# Preprocessing the dataset to align NER tags with tokens
label_list = ["O", "B-PER", "I-PER", "B-ORG", "I-ORG", "B-LOC", "I-LOC"]

def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    previous_word_id = None
    for word_id in word_ids:
        if word_id is None:
            new_labels.append(-100)  # Ignore these tokens in the loss
        elif word_id != previous_word_id:
            new_labels.append(labels[word_id])  # Only label first token of the word
        else:
            # Continue with I- tags for inner tokens, otherwise ignore
            new_labels.append(labels[word_id] if label_list[labels[word_id]].startswith("I-") else -100)
        previous_word_id = word_id
    return new_labels


def preprocess_data(batch):
    tokenized_inputs = tokenizer(batch["tokens"], truncation=True, is_split_into_words=True, padding="max_length", max_length=128)
    all_word_ids = [tokenized_inputs.word_ids(i) for i in range(len(batch["tokens"]))]

    # Align labels with word ids for each example in the batch
    all_aligned_labels = [
        align_labels_with_tokens(labels, word_ids) for labels, word_ids in zip(batch["ner_tags"], all_word_ids)
    ]

    # Add aligned labels to tokenized inputs
    tokenized_inputs["labels"] = all_aligned_labels
    return tokenized_inputs


train_dataset = train_data.map(preprocess_data, batched=True)
validation_dataset = validation_data.map(preprocess_data, batched=True)
test_dataset = test_data.map(preprocess_data, batched=True)

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

# Load the model with more capacity for multilingual tasks
# Load model
model = AutoModelForTokenClassification.from_pretrained("l3cube-pune/tamil-bert", num_labels=len(label_list))

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

In [ ]:
!pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=c230215bcfdfb6ce7b31a5d570c65d166a3fd0c49e00700864e7f64ad3b0f19f
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [ ]:
from seqeval.metrics import classification_report, f1_score, accuracy_score, precision_score, recall_score
import numpy as np

# Define compute_metrics function
def compute_metrics(pred):
    label_ids = pred.label_ids
    predictions = pred.predictions.argmax(-1)

    # Remove ignored index (special tokens)
    true_labels = [[label_list[label_id] for (label_id, pred_id) in zip(label_ids[i], predictions[i]) if label_id != -100]
                   for i in range(len(label_ids))]
    true_predictions = [[label_list[pred_id] for (label_id, pred_id) in zip(label_ids[i], predictions[i]) if label_id != -100]
                        for i in range(len(predictions))]

    # Calculate metrics using seqeval
    precision = precision_score(true_labels, true_predictions)
    recall = recall_score(true_labels, true_predictions)
    f1 = f1_score(true_labels, true_predictions)
    accuracy = accuracy_score(true_labels, true_predictions)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [ ]:
# A-LLRD Optimizer
def get_optimizer(model, base_lr=2e-5, min_decay=0.8, max_decay=0.95):
    def compute_gradient_norms(model):
        norms = []
        for i, layer in enumerate(model.bert.encoder.layer):
            norm = 0
            for param in layer.parameters():
                if param.grad is not None:
                    norm += param.grad.norm().item() ** 2
            norms.append(np.sqrt(norm) if norm > 0 else 1e-8)  # Avoid division by zero
        return norms

    class ALLRDOptimizer(AdamW):
        def __init__(self, params, model, base_lr, min_decay, max_decay, betas=(0.9, 0.999), eps=1e-8, amsgrad=False, weight_decay=0.01, fused=False, capturable=False, differentiable=False, maximize=False, foreach=None):
            defaults = dict(
                lr=base_lr,
                betas=betas,
                eps=eps,
                amsgrad=amsgrad,
                weight_decay=weight_decay,
                fused=fused,
                capturable=capturable,
                differentiable=differentiable,
                maximize=maximize,
                foreach=foreach
            )
            super().__init__(params, **defaults)
            self.model = model
            self.base_lr = base_lr
            self.min_decay = min_decay
            self.max_decay = max_decay
            self.step_count = 0

        def step(self, closure=None):
            self.step_count += 1
            # Compute gradient norms
            norms = compute_gradient_norms(self.model)
            # Clip norms to prevent instability
            norms = [min(max(norm, 1e-8), 10.0) for norm in norms]
            # Softmax over negative norms to get decay factors
            norms = torch.tensor(norms, device=next(self.model.parameters()).device)
            decay_factors = torch.softmax(-norms, dim=0).cpu().numpy()
            # Scale decay factors to [min_decay, max_decay]
            decay_factors = self.min_decay + (self.max_decay - self.min_decay) * decay_factors / decay_factors.max()
            # Log decay factors for debugging
            if self.step_count % 100 == 0:
                print(f"Step {self.step_count}: Decay factors: {decay_factors.tolist()}")
            # Update learning rates
            param_groups = []
            for i, layer in enumerate(self.model.bert.encoder.layer):
                layer_lr = self.base_lr * decay_factors[i]
                param_groups.append({
                    "params": layer.parameters(),
                    "lr": layer_lr,
                    "betas": self.defaults["betas"],
                    "eps": self.defaults["eps"],
                    "amsgrad": self.defaults["amsgrad"],
                    "weight_decay": self.defaults["weight_decay"],
                    "fused": self.defaults["fused"],
                    "capturable": self.defaults["capturable"],
                    "differentiable": self.defaults["differentiable"],
                    "maximize": self.defaults["maximize"],
                    "foreach": self.defaults["foreach"]
                })
            param_groups.append({
                "params": self.model.classifier.parameters(),
                "lr": self.base_lr,
                "betas": self.defaults["betas"],
                "eps": self.defaults["eps"],
                "amsgrad": self.defaults["amsgrad"],
                "weight_decay": self.defaults["weight_decay"],
                "fused": self.defaults["fused"],
                "capturable": self.defaults["capturable"],
                "differentiable": self.defaults["differentiable"],
                "maximize": self.defaults["maximize"],
                "foreach": self.defaults["foreach"]
            })
            self.param_groups = param_groups
            super().step(closure)

    optimizer_grouped_parameters = [
        {
            "params": layer.parameters(),
            "lr": base_lr,
            "betas": (0.9, 0.999),
            "eps": 1e-8,
            "amsgrad": False,
            "weight_decay": 0.01,
            "fused": False,
            "capturable": False,
            "differentiable": False,
            "maximize": False,
            "foreach": None
        } for layer in model.bert.encoder.layer
    ] + [{
        "params": model.classifier.parameters(),
        "lr": base_lr,
        "betas": (0.9, 0.999),
        "eps": 1e-8,
        "amsgrad": False,
        "weight_decay": 0.01,
        "fused": False,
        "capturable": False,
        "differentiable": False,
        "maximize": False,
        "foreach": None
    }]
    return ALLRDOptimizer(optimizer_grouped_parameters, model, base_lr, min_decay, max_decay)

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="no",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    report_to="none",
)

# Initialize trainer
data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    optimizers=(get_optimizer(model), None)
)

# Train and evaluate
trainer.train()
metrics = trainer.evaluate(test_dataset)
metrics_df = pd.DataFrame([metrics])
print(metrics_df)

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.863200,0.571782,0.949000,0.828412,0.850820,0.839466
2,0.307900,0.258080,0.955298,0.841017,0.867213,0.853914
3,0.166400,0.166071,0.964065,0.865186,0.894262,0.879484
4,0.108500,0.156698,0.964806,0.870079,0.905738,0.887550
5,0.078200,0.158504,0.965918,0.886473,0.902459,0.894395
6,0.056800,0.175775,0.964189,0.869976,0.904918,0.887103
7,0.051700,0.192072,0.964683,0.887470,0.911475,0.899313
8,0.043200,0.201638,0.966041,0.892456,0.911475,0.901865
9,0.028900,0.212569,0.967029,0.892971,0.916393,0.904531
10,0.020900,0.240079,0.964559,0.876875,0.910656,0.893446


Step 100: Decay factors: [0.95, 0.9489958034469612, 0.9493729916703832, 0.9476926810422204, 0.9493745353999579, 0.949278758148247, 0.9490666394930805, 0.949035063464093, 0.9492496488304083, 0.9493728278915163, 0.9493854044089326, 0.9003212687982676]
Step 200: Decay factors: [0.9452577201584095, 0.9421556705696664, 0.9432065062108934, 0.943831442561651, 0.9438402084546126, 0.9442899969426748, 0.9452913073394507, 0.9462971531935934, 0.9478678077747744, 0.9485284102176772, 0.95, 0.8930745830260554]
Step 300: Decay factors: [0.9486069359499134, 0.9472860404048776, 0.9469842905642829, 0.9469250031338083, 0.9477744570346995, 0.9478237947368289, 0.9489189072493167, 0.9498017410037863, 0.95, 0.9498483258889395, 0.9471595842223349, 0.8926263894783727]
Step 400: Decay factors: [0.9281733735862192, 0.9271962304155921, 0.9297545672712172, 0.9311213093095447, 0.9319489645199526, 0.934376247801477, 0.9403429415389942, 0.9440380440955773, 0.9475762262948558, 0.9470431300294867, 0.95, 0.88356534712976

   eval_loss  eval_accuracy  eval_precision  eval_recall   eval_f1  \
0   0.227002       0.966194        0.883503     0.910556  0.896825   

   eval_runtime  eval_samples_per_second  eval_steps_per_second  epoch  
0        4.3421                  230.302                 28.788   10.0  


In [ ]:
# Inspect the log history to understand the structure of logs
for log in trainer.state.log_history:
    print(log)


{'loss': 1.6467, 'grad_norm': 4.213992595672607, 'learning_rate': 1.9467733333333333e-05, 'epoch': 0.26666666666666666, 'step': 500}
{'loss': 1.1948, 'grad_norm': 2.2808005809783936, 'learning_rate': 1.8934400000000003e-05, 'epoch': 0.5333333333333333, 'step': 1000}
{'loss': 0.8632, 'grad_norm': 2.366084098815918, 'learning_rate': 1.8401066666666666e-05, 'epoch': 0.8, 'step': 1500}
{'eval_loss': 0.5717816352844238, 'eval_accuracy': 0.9489997530254384, 'eval_precision': 0.8284118116520351, 'eval_recall': 0.8508196721311475, 'eval_f1': 0.8394662353416903, 'eval_runtime': 4.3529, 'eval_samples_per_second': 229.73, 'eval_steps_per_second': 28.716, 'epoch': 1.0, 'step': 1875}
{'loss': 0.6443, 'grad_norm': 2.159210681915283, 'learning_rate': 1.7867733333333335e-05, 'epoch': 1.0666666666666667, 'step': 2000}
{'loss': 0.4826, 'grad_norm': 2.5302393436431885, 'learning_rate': 1.73344e-05, 'epoch': 1.3333333333333333, 'step': 2500}
{'loss': 0.3915, 'grad_norm': 1.135233998298645, 'learning_rate'

In [ ]:
# Log validation metrics per epoch
print("\nValidation Metrics per Epoch:")
validation_metrics = []
for log in trainer.state.log_history:
    if "eval_f1" in log:  # Validation metrics
        epoch_metrics = {
            "epoch": log.get("epoch", None),
            "eval_loss": log.get("eval_loss", None),
            "eval_accuracy": log.get("eval_accuracy", None),
            "eval_precision": log.get("eval_precision", None),
            "eval_recall": log.get("eval_recall", None),
            "eval_f1": log.get("eval_f1", None)
        }
        validation_metrics.append(epoch_metrics)
validation_df = pd.DataFrame(validation_metrics)
print(validation_df)

# Evaluate on test dataset (overall metrics)
print("\nTest Metrics:")
test_metrics = trainer.evaluate(test_dataset)
test_metrics_summary = {
    "test_accuracy": test_metrics["eval_accuracy"],
    "test_precision": test_metrics["eval_precision"],
    "test_recall": test_metrics["eval_recall"],
    "test_f1": test_metrics["eval_f1"]
}
test_metrics_df = pd.DataFrame([test_metrics_summary])
print(test_metrics_df)

# Compute per-class metrics manually
print("\nPer-Class Test Metrics:")
model.eval()
all_true_labels = []
all_pred_labels = []

# Run predictions on test dataset
for batch in trainer.get_test_dataloader(test_dataset):
    batch = {k: v.to(model.device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    predictions = outputs.logits.argmax(dim=-1).cpu().numpy()
    labels = batch["labels"].cpu().numpy()

    # Collect true and predicted labels, ignoring -100
    for i in range(len(labels)):
        true_labels = [label_list[l] for l, p in zip(labels[i], predictions[i]) if l != -100]
        pred_labels = [label_list[p] for l, p in zip(labels[i], predictions[i]) if l != -100]
        all_true_labels.extend(true_labels)
        all_pred_labels.extend(pred_labels)

# Compute classification report
class_report = classification_report([all_true_labels], [all_pred_labels], output_dict=True)
per_class_data = []
for label, metrics in class_report.items():
    if label in label_list:  # Only include actual NER labels
        per_class_data.append({
            "label": label,
            "precision": metrics["precision"],
            "recall": metrics["recall"],
            "f1-score": metrics["f1-score"],
            "support": metrics["support"]
        })
per_class_df = pd.DataFrame(per_class_data)
print(per_class_df)


Validation Metrics per Epoch:
    epoch  eval_loss  eval_accuracy  eval_precision  eval_recall   eval_f1
0     1.0   0.571782       0.949000        0.828412     0.850820  0.839466
1     2.0   0.258080       0.955298        0.841017     0.867213  0.853914
2     3.0   0.166071       0.964065        0.865186     0.894262  0.879484
3     4.0   0.156698       0.964806        0.870079     0.905738  0.887550
4     5.0   0.158504       0.965918        0.886473     0.902459  0.894395
5     6.0   0.175775       0.964189        0.869976     0.904918  0.887103
6     7.0   0.192072       0.964683        0.887470     0.911475  0.899313
7     8.0   0.201638       0.966041        0.892456     0.911475  0.901865
8     9.0   0.212569       0.967029        0.892971     0.916393  0.904531
9    10.0   0.240079       0.964559        0.876875     0.910656  0.893446
10   10.0   0.227002       0.966194        0.883503     0.910556  0.896825
11   10.0   0.227002       0.966194        0.883503     0.910556  0.8

In [ ]:
# Test model on example sentences
def test_model(sentences):
    model.eval()
    for sentence in sentences:
        tokenized_input = tokenizer(sentence, truncation=True, padding="max_length", max_length=128, return_tensors="pt")
        input_ids = tokenized_input["input_ids"].to(model.device)
        attention_mask = tokenized_input["attention_mask"].to(model.device)
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
        predictions = np.argmax(outputs.logits.cpu().numpy(), axis=2)
        tokens = tokenizer.convert_ids_to_tokens(input_ids[0].cpu().numpy())
        predicted_labels = [label_list[p] for p in predictions[0]]
        print(f"\nTesting sentence: {sentence}")
        for token, label in zip(tokens, predicted_labels):
            if token not in tokenizer.special_tokens_map.values() and token.strip():
                print(f"Word: '{token}', Predicted Label: {label}")

test_sentences = [
    "என் பெயர் வின்ஸ்டன் சர்ச்சில் மற்றும் நான் கோயம்புத்தூரில் வருகிறேன்.",
    "இது ஒரு புதிய தொழில்நுட்ப மேம்பாடு.",
    "தமிழ் மொழி தமிழ்நாட்டின் நிலையான மொழியாகும்.",
    "இத்தாவரம் சீனா, இந்தியா, ஜப்பான், பப்பாசியா, மைக்குரோனீசியா போன்ற நாடுகளில் காணப்படுகிறது.",
    "சைஃப் அலி கான், தீபிகா படுகோண், ஜான் ஆபிரகாம் (நடிகர்), ஜாக்குலின் பெர்னாண்டஸ், அனில் கபூர், அமீஷா பட்டேல், மற்றும் ரஜினிகாந்த் ஆகியோர்."
]
test_model(test_sentences)


Testing sentence: என் பெயர் வின்ஸ்டன் சர்ச்சில் மற்றும் நான் கோயம்புத்தூரில் வருகிறேன்.
Word: 'என்', Predicted Label: O
Word: 'பெயர்', Predicted Label: O
Word: 'வின்', Predicted Label: B-PER
Word: '##ஸ்டன்', Predicted Label: B-PER
Word: 'சர்ச்', Predicted Label: I-PER
Word: '##சில்', Predicted Label: I-PER
Word: 'மற்றும்', Predicted Label: O
Word: 'நான்', Predicted Label: O
Word: 'கோ', Predicted Label: B-LOC
Word: '##யம்', Predicted Label: B-LOC
Word: '##புத்', Predicted Label: I-LOC
Word: '##தூரில்', Predicted Label: I-LOC
Word: 'வருகிறேன்', Predicted Label: O
Word: '.', Predicted Label: O

Testing sentence: இது ஒரு புதிய தொழில்நுட்ப மேம்பாடு.
Word: 'இது', Predicted Label: O
Word: 'ஒரு', Predicted Label: O
Word: 'புதிய', Predicted Label: O
Word: 'தொழில்நுட்ப', Predicted Label: O
Word: 'மேம்பாடு', Predicted Label: O
Word: '.', Predicted Label: O

Testing sentence: தமிழ் மொழி தமிழ்நாட்டின் நிலையான மொழியாகும்.
Word: 'தமிழ்', Predicted Label: B-ORG
Word: 'மொழி', Predicted Label: I-ORG
Wo